# Modelling TOI-1233 with `chexoplanet`

This tutorial is a good starting point to understand the `chexoplanet` code.

Let's import some modules:

In [1]:
from chexoplanet import newfit
import numpy as np
import pandas as pd

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


#### Initialising the model
To set up a chexoplanet model, you need to give it a name. If possible, use the same name which is found in the CHEOPS database (in this case, TOI1233 without a hyphen).

It is worth including a comment specific to this model run, which will enable you to check between different model runs later on.

You may also decide to specify the RA/Dec of the target here (using `astropy.coordinates.SkyCoord` object), which might help to search later.

In [ ]:
mod = newfit.chexo_model("TOI1233", comment="example_fit")

#### Adding a TESS lightcurve

`chexoplanet` was specifically build to interact with data from other exoplanet missions natively, as almost all targets will have for example archival TESS photometry.

The easiest way to access archival data is to use `get_tess` which piggy-backs on the functionality of `MonoTools.lightcurve`. This automatically searches Kepler, K2 and TESS data, downloading all available lightcurves:

In [ ]:
mod.get_tess(tic=260647166)

However, `MonoTools` is not the easiest to install, and `chexoplanet` also allows you to add lightcurves directly by specifying a time, flux and flux_err arrays to `add_lc`. Flux and errors should be in ppt. It's necessary, especially when there are multiple data sources (e.g. K2 & TESS), to also include the name of the lightcurve - in this case 'tess'.

In either case, `chexoplanet` will initialise lightcurves at the location of `mod.lcs[src]` where src is e.g. 'tess'.

In [ ]:
##Commented out as this would duplicate the line above
#tess_lc=pd.read_csv("dummy_lightcurve.csv")
#mod.add_lc(time=tess_lc['time'], flux=(tess_lc['flux']-1.0)*1000, flux_err=tess_lc['flux_err']*1000, src='tess')

#### Initialising Stellar Parameters

Once again, this can be automatically read from the TIC using the data scraped by `MonoTools.lightcurve`. This is automatically done in `get_tess`.

Alternatively stellar parameters can be added manually by giving `[value, neg_err, pos_err]` lists to `init_starpars`.

In [ ]:
mod.init_starpars(Rstar=[0.876, 0.007, 0.007],
                  Mstar=[0.897, 0.042, 0.042],
                  Teff=[5660, 61, 61],
                  logg=[4.49, 0.11, 0.11])

#### Adding CHEOPS observations

The simplest way to add a cheops observation is to use `mod.add_cheops_lc()`, specifying the precise filekey wanted.

If `download=True` then the data will be downloaded from Dace. In order to get private data, make sure you have a DACE log-in, have generated a DACE access token which is in your `.dacerc` file and, bizarrely, make sure you are actively logged into Dace on your browser (even then, this API access step may require a couple of attempts).

You can also specify whether to use the standard "Data Reduction Pipeline" output (`DRP=True`) or whether to perform PSF modelling (using `PIPE=True`).

In the case of PIPE data, which I recommend using, make sure that PIPE is downloaded from github at the most up-to-date version (you may require the `BG-star-fits` branch), that it has been installed via `pip install .` in the PIPE directory, and and that all the necessary config files have been downloaded (these do not come with the github package and you may need to contact PIPE creator Alexis Brandeker). Running PIPE may be time-consuming and take 10-20mins per filekey.

In [ ]:
# # Commented out - you will see why in a second
# mod.add_cheops_lc('CH_PR110045_TG002701_V0200',DRP=False,PIPE=True)

However, the easiest way to do all of this is to use `mod.get_cheops()`. This will perform the above step for all CHEOPS data found on DACE.

In [ ]:
mod.get_cheops()

#### Adding planets to a model

There are, as before, two ways to add planets to a model:

1) The precise way (using `mod.add_planet()`) - Here you should specify the planet name and then initial values for the modelling, namely: `tcen` (transit epoch; same units as the data in `mod.lcs`), `tcen_err`, `tdur` (duration in days), `depth` (as a ratio), `period` and `period_err` (in days).

2) The lazy way (using `mod.add_planets_from_toi()`) - this scrapes the planetary info from the TOI catalogue in order to add them directly to the model.

In [ ]:
mod.add_planets_from_toi()
mod.add_planet("f", tcen=1793.2786,tcen_err=0.01,tdur=3.27/24,depth=480,period=29.54115,period_err=1e-4)

### Initialising the TESS lightcurve

In [ ]:
mod.init_lc(fit_gp=False, fit_flat=True, cut_oot=True, bin_oot=False)

### Initialising the CHEOPS lightcurve

In [ ]:
mod.init_cheops(use_bayes_fact=True, use_signif=False, overwrite=False)

### Initialising the full model

mod.init_model(use_mstar=False, use_logg=True,fit_phi_spline=True,fit_phi_gp=False, constrain_lds=True, fit_ttvs=False, assume_circ=False)

### Plotting

In [ ]:
mod.plot_phot(src='tess',save=True)
mod.plot_phot(src='k2',save=True)
mod.plot_cheops(save=True,show_detrend=True)
mod.plot_transits_fold(save=True)

### Sampling

In [ ]:
mod.sample_model()

### Saving

In [ ]:
mod.save_timeseries()
mod.save_model_to_file()